# Minimize example - Lammps

In [1]:
# headers

# general modules
import numpy as np
import matplotlib.pyplot as plt

# pyiron modules
from pyiron_atomistics import Project
import pyiron_contrib

In [2]:
# define project

pr = Project('minimize_example')
pr.remove_jobs_silently(recursive=True)

In [3]:
# inputs 

# structure specific
element = 'Al'
supercell = 3
cubic = True
a = 4.  # we purposefully set this value off the actual a value

# job specific
potential = '2008--Mendelev-M-I--Al--LAMMPS--ipr1'

# Minimize specific
n_steps = 200
gamma0 = 0.1
f_tol = 1e-4
dynamic_gamma = True

In [4]:
# create base structure

box = pr.create.structure.bulk(name=element, cubic=cubic, a=a).repeat(supercell)
# distort the structure, so the effects of minimization can be seen
box.positions[0] += 0.5

In [5]:
# create and run the minimize job

pr_prot = pr.create_group('protocol_mini')  # create a new folder

mini_ref = pr_prot.create.job.Lammps(job_name='ref_mini')
mini_ref.structure = box
mini_ref.potential = potential
mini_ref.save()  # Don't forget this step!

prot_mini_job = pr_prot.create.job.ProtoMinimGradDes('prot_mini_job')
prot_mini_job.input.ref_job_full_path = mini_ref.path
prot_mini_job.input.structure = box
prot_mini_job.input.n_steps = n_steps
prot_mini_job.input.f_tol = f_tol
prot_mini_job.input.gamma0 = gamma0
prot_mini_job.input.dynamic_gamma = dynamic_gamma

# # in addition to these inputs, you can also modify the following inputs
# prot_mini_job.input.fix_com = True
# prot_mini_job.input.c = 0.1
# prot_mini_job.input.tau1 = 1.
# prot_mini_job.input.tau2 = 0.2

prot_mini_job.run()

The job ref_mini was saved and received the ID: 107
The job prot_mini_job was saved and received the ID: 108
The job initialize_job_0 was saved and received the ID: 111


/home/raynol025/pyiron/pyiron_base/pyiron_base/generic/parameters.py:282: UserWarning: The input in GenericParameters changed, while the state of the job was already finished.
  warnings.warn(


In [6]:
# check what keys are in the output dictionary

prot_mini_job.output.keys()

dict_keys(['energy_pot', 'max_force', 'positions', 'forces'])

In [7]:
# run pyiron's lammps minimize

pr_pyir = pr.create_group('pyiron_mini')

pyir_mini_job = pr_pyir.create.job.Lammps('pyir_mini_job')
pyir_mini_job.structure = box
pyir_mini_job.potential = potential
pyir_mini_job.calc_minimize(pressure=None, style='sd', ionic_force_tolerance = f_tol)
# since this protocol method is a dummy method to demonstrate how a protocol works, 
# it presently can only relax ionic positions, and not the simulation cell. For this reason,
# we keep pressure=None (isochoric relaxation).

# the protocol uses the gradient descent method for the minimization of forces. 
# We select the steepest descent method for the minimization in the lammps job.
pyir_mini_job.run()

The job pyir_mini_job was saved and received the ID: 113


/home/raynol025/pyiron/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/raynol025/pyiron/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/raynol025/pyiron/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/raynol025/pyiro

In [8]:
# compare the energy_pots from the 2 methods

prot_mini_job.output.energy_pot[-1], pyir_mini_job.output.energy_pot[-1]

(-367.8223718746988, -367.822371885738)